In [ ]:
# !pip install -r requirements.txt

In [1]:
import csv
import cv2
import numpy as np
import os
from shapely.geometry import Polygon
from shapely.geometry.point import Point
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors

In [2]:
model_path = "model_training/runs/detect/train/weights/best.pt"
model = YOLO(model_path)
image_dir = "input_images/"
output_dir = "output_images/"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Define detection region points
inset = 300
file0 = os.listdir(image_dir)[0]
image_path = os.path.join(image_dir, file0)
image0 = cv2.imread(image_path)
h, w = image0.shape[:2]
region_points = np.array([[inset, inset], [w - inset, inset], [w - inset, h - inset], [inset, h - inset]], np.int32)

In [3]:
with open('counts.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Aliquot', 'Larvae', 'Eggs'])

    for filename in os.listdir(image_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(image_dir, filename)
            image      = cv2.imread(image_path)
    
            results = model.track(image, verbose=False)
            boxes   = results[0].boxes.xywh.cpu()
            clss    = results[0].boxes.cls.cpu().tolist()
            names   = results[0].names
    
            larvae = 0
            eggs   = 0  
            
            annotator = Annotator(image, line_width=2, example=str(names))
    
            for box, cls in zip(boxes, clss):
                x, y, w, h = box
                label = str(names[cls])
                xyxy = (x - w / 2), (y - h / 2), (x + w / 2), (y + h / 2)
    
                bbox_color = colors(cls, True)
                annotator.box_label(xyxy, label, color=bbox_color)
    
                if Polygon(region_points).contains(Point((x, y))):
                    if cls == 0:
                        larvae += 1
                    elif cls == 1:
                        eggs +=1
    
            cv2.polylines(image, [region_points], True, (255, 0, 0), 2)

            index = filename.find(".")
            aliquot = filename[:index]
            writer.writerow([aliquot, larvae, eggs])
            print(f"{aliquot}\nlarvae: {larvae}\neggs  : {eggs}\n")
                                      
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, image)
print("Object counting completed.")


A6
larvae: 4
eggs  : 2

D8
larvae: 0
eggs  : 3

G7
larvae: 0
eggs  : 0

H2
larvae: 2
eggs  : 6

H12
larvae: 1
eggs  : 2

B1
larvae: 2
eggs  : 1

D4
larvae: 2
eggs  : 1

A2
larvae: 2
eggs  : 2

E2
larvae: 3
eggs  : 0

H3
larvae: 2
eggs  : 1

Object counting completed.
